In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 9)
plt.style.use('seaborn-darkgrid')
import seaborn as sns
sns.set_style('darkgrid')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import IntegerLookup

from tqdm.keras import TqdmCallback
from tqdm.auto import tqdm
import ubiquant
import gc

In [ ]:
train = pd.read_pickle('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl')

investment_ids = train['investment_id'].unique()
investment_size = train['investment_id'].nunique() + 1

InvestmentLayer = IntegerLookup(max_tokens = investment_size)
InvestmentLayer.adapt(
    pd.DataFrame({
        'investment_ids' : investment_ids.tolist()
    })
)

In [ ]:
env = ubiquant.make_env()

In [ ]:
iter_test = env.iter_test()

In [ ]:
def make_dataset0(x, y = None, batch_size = 512, shuffle = False) :
    def preprocess(x, y) :
        return x, y
    feat = [col for col in x.columns if col.startswith('f_')]
    feature = x[feat].values
    dataset = tf.data.Dataset.from_tensor_slices((
        feature, y
    )).map(preprocess).batch(batch_size = batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    if shuffle :
        dataset = dataset.shuffle(500, seed = 50)
    return dataset

In [ ]:
def make_dataset1(x, y = None, batch_size = 512, shuffle = False) :
    def preprocess(x, y) :
        return x, y
    feat = [col for col in x.columns if col.startswith('f_')]
    investment = x['investment_id'].values
    feature = x[feat].values
    dataset = tf.data.Dataset.from_tensor_slices((
        (investment, feature), y
    )).map(preprocess).batch(batch_size = batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    if shuffle :
        dataset = dataset.shuffle(500, seed = 50)
    return dataset

In [ ]:
try :
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver().connect()
    strategy = tf.distribute.TPUStrategy(tpu)
except :
    strategy = tf.distribute.get_strategy()

In [ ]:
def correlationMetric(x, y, axis=-2):
    from tensorflow.python.ops import math_ops
    """Metric returning the Pearson correlation coefficient of two tensors over some axis, default -2."""
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xvar = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    yvar = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xvar * yvar)
    return corr


def correlationLoss(x,y, axis=-2):
    from tensorflow.python.ops import math_ops
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xsqsum = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    ysqsum = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xsqsum * ysqsum)
    return tf.convert_to_tensor( K.mean(tf.constant(1.0, dtype=x.dtype) - corr ) , dtype=tf.float32 )

def build_model0() :
    with strategy.scope() :
        inputs = keras.layers.Input(shape = (300, ), dtype = tf.float16)
        x = keras.layers.Dense(256, activation = 'swish')(inputs)
        x = keras.layers.Dropout(.1)(x)

        x = keras.layers.Reshape((-1, 1))(x)
        x = keras.layers.Conv1D(16, 4, strides = 1, padding = 'same')(x)
        x = keras.layers.MaxPool1D()(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Conv1D(32, 4, padding = 'same')(x)
        x = keras.layers.MaxPool1D()(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Conv1D(64, 4, padding = 'same')(x)
        x = keras.layers.MaxPool1D()(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Conv1D(128, 4, padding = 'same')(x)
        x = keras.layers.MaxPool1D()(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Conv1D(128, 4, padding = 'same')(x)
        x = keras.layers.MaxPool1D()(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Flatten()(x)

        x1 = keras.layers.Dense(256, activation = 'swish', kernel_regularizer = 'l2')(x)
        x2 = keras.layers.Dense(128, activation = 'swish', kernel_regularizer=  'l2')(x1)

        x = keras.layers.Concatenate(axis = -1)([x1, x2])
        x = keras.layers.Dense(64, activation = 'swish', kernel_regularizer = 'l2')(x)
        out = keras.layers.Dense(1, activation = 'linear')(x)

        model = keras.models.Model(
            inputs = inputs,
            outputs = out
        )
        model.compile(
            optimizer = keras.optimizers.Adam(learning_rate = 7e-4),
            loss = correlationLoss,
            metrics = [correlationMetric, keras.metrics.RootMeanSquaredError(name = 'rmse')]
        )
    return model

In [ ]:
def build_model1() :
    investment = keras.layers.Input(shape = (1, ), dtype = tf.uint16)
    xinvest = InvestmentLayer(investment)
    xinvest = keras.layers.Embedding(investment_size, 64, input_length = 1)(xinvest)
    xinvest = keras.layers.Reshape((-1,))(xinvest)
    
    xinvest = keras.layers.Dense(128, activation = 'swish')(xinvest)
    xinvest = keras.layers.Dense(128, activation = 'swish')(xinvest)
    xinvest = keras.layers.Dense(256, activation = 'swish')(xinvest)
    
    feat = keras.layers.Input(shape = (300, ), dtype = tf.float16)
    xfeat = keras.layers.BatchNormalization()(feat)
    xfeat = keras.layers.Dense(512, activation = 'swish')(xfeat)
    xfeat = keras.layers.Reshape((-1, 1))(xfeat)
    
    xfeat = keras.layers.Conv1D(16, 4)(xfeat)
    xfeat = keras.layers.MaxPool1D()(xfeat)
    xfeat = keras.layers.BatchNormalization()(xfeat)
    
    xfeat = keras.layers.Conv1D(32, 4)(xfeat)
    xfeat = keras.layers.MaxPool1D()(xfeat)
    xfeat = keras.layers.BatchNormalization()(xfeat)
    
    xfeat = keras.layers.Conv1D(32, 4)(xfeat)
    xfeat = keras.layers.MaxPool1D()(xfeat)
    xfeat = keras.layers.BatchNormalization()(xfeat)
    
    xfeat = keras.layers.Conv1D(64, 4)(xfeat)
    xfeat = keras.layers.MaxPool1D()(xfeat)
    xfeat = keras.layers.BatchNormalization()(xfeat)
    
    xfeat = keras.layers.Conv1D(64, 4)(xfeat)
    xfeat = keras.layers.MaxPool1D()(xfeat)
    xfeat = keras.layers.BatchNormalization()(xfeat)
    
    xfeat = keras.layers.Conv1D(128, 4)(xfeat)
    xfeat = keras.layers.MaxPool1D()(xfeat)
    xfeat = keras.layers.BatchNormalization()(xfeat)
    
    xfeat = keras.layers.Conv1D(256, 4)(xfeat)
    xfeat = keras.layers.MaxPool1D()(xfeat)
    xfeat = keras.layers.BatchNormalization()(xfeat)
    
    xfeat = keras.layers.Flatten()(xfeat)
    
    x = keras.layers.Concatenate(axis = -1)([xinvest, xfeat])
    
    x1 = keras.layers.Dense(512, activation = 'swish', kernel_regularizer = 'l2')(x)
    x2 = keras.layers.Dense(512, activation = 'swish', kernel_regularizer = 'l2')(x)
    
    x = keras.layers.Concatenate(axis = -1)([x1, x2, x])
    
    x3 = keras.layers.Dense(256, activation = 'swish', kernel_regularizer = 'l2')(x)
    x4 = keras.layers.Dense(256, activation = 'swish', kernel_regularizer = 'l2')(x)
    
    x = keras.layers.Concatenate(axis = -1)([x3, x4, x])
    
    x5 = keras.layers.Dense(128, activation = 'swish', kernel_regularizer = 'l2')(x)
    x6 = keras.layers.Dense(128, activation = 'swish', kernel_regularizer = 'l2')(x)
    
    x = keras.layers.Concatenate(axis = -1)([x1, x2, x3, x4, x5, x6])
    x = keras.layers.Dense(64, activation = 'swish', kernel_regularizer = 'l2')(x)
    
    x = keras.layers.Dense(1, activation = 'linear')(x)
    
    model = keras.models.Model(
        inputs = [investment, feat],
        outputs = x
    )
    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = 7e-4),
        loss = correlationLoss,
        metrics = [correlationMetric]
    )
    return model

In [ ]:
def load_and_save_model(model, weights) :
    model.load_weights(weights)
    return model

In [ ]:
model0 = build_model0()
models0 = [
    load_and_save_model(model0, f'../input/cnn-just-feature/cnn_model_fold_{i}')
    for i in range(4)
]
model1 = build_model1()
models1 = [
    load_and_save_model(model1, f'../input/cnn-train/model_fold{i}')
    for i in range(4)
]

In [ ]:
%%time
for i, (test, sub) in enumerate(iter_test) :
    try :
        preds = []
        test_ds0 = make_dataset0(test, batch_size = 512, shuffle = False)
        for model in models0 :
            preds.append(model.predict(test_ds0).ravel())
        test_ds1 = make_dataset1(test, batch_size = 512, shuffle = False)
        for model in models1 :
            preds.append(model.predict(test_ds1).ravel())
        preds = np.max(preds, axis = 0)
        sub['target'] = preds
        del test_ds0, test_ds1
    except :
        sub['target'] = 0
        print(f'Exception in iter no. {i}')
    if i == 0 :
        display(sub)
    env.predict(sub)
    K.clear_session()
    gc.collect()